In [268]:
import math
import scipy
import numpy as np
import matplotlib.pyplot as plt
import datetime

# This program simulates demand data for the inventory management system. It must be run for one SKU at a time

"""
Plan:

Define five cases for demand data distributions

Case 1: Uniform Sales
- Roughly constant sales throughout the year with some variation, but no seasonal spikes

Case 2: Bimodal Sales
- Product follows uniform distribution for most of the year but spikes around Christmas and Thanksgiving

Case 3: Pronounced Seasonality
- Product experiences low sales for much of the year but exhibits a profound spike in a given season

Case 4: Trigonometric Variation
- Product demand varies trigonometrically 
- This is commonly observed in fashion, FMCG, and other cyclical industries

Case 5: Rising star
- Product exhibits demand growth over time with no sign of slowing down
- This will not follow a pure exponential growth pattern, but there needs to be a clear upward trend

Simulate sales, prices, quantities, dates, and times for each item

Plot it and print the case number as a verification mechanism

"""
#######################################
#  STEP 1: SET SIMULATION PARAMETERS  #
#######################################

# SET PRODUCT TO BE SIMULATED AND ITS ATTRIBUTES
SKU = str(input("Enter the SKU to be simulated. "))
name = str(input("Enter the name of the product to be simulated. "))
MSRP = 100 # this will be used to define price variation (assume no sales above MSRP but stochastic variation in discounts)

# SET DESIRED NUMBER OF DATA POINTS
data_points = 3000

# SET TIME INTERVAL TO SIMULATE
start_date = datetime.date(2024,1,1)
end_date = datetime.date(2025,12,31)

# SET PARAMETERS FOR EACH DEMAND CASE
# scale factors define demand variation for each case - larger values mean more fluctuation
# discount propensity defines frequency of sale below MSRP - this parameter CANNOT exceed MSRP!
case1_scale_factor = 100
case1_discount_propensity = 20 

case2_scale_factor = 150
case2_discount_propensity = 30

case3_scale_factor = 200
case3_discount_propensity = 10
# seasons: spring is March-May, summer is June-August, fall is September-November, winter is December-February
case3_season = int(input("CHOOSE CASE 3 SEASON: Enter 1 for spring, 2 for summer, 3 for fall, or 4 for winter: "))

case4_scale_factor = 125
case4_discount_propensity = 14

case5_scale_factor = 1250
case5_discount_propensity = 45
case5_growth_rate = 0.001

num_days = (end_date - start_date).days    # do not modify

outfilename = str(input("Enter the output file name. "))

###########################################################
#  STEP 2: SYSTEM CHOOSES A RANDOM DEMAND CASE AT RANDOM  #
###########################################################

cases = [1,2,3,4,5]
# case = 5
case = math.ceil(5*np.random.rand())

####################################
#  STEP 3: INITIALIZE DATA ARRAYS  #
####################################

timeintarray = np.linspace(0, num_days, data_points)
dates = np.empty(data_points, dtype = object) # initialize date array and ensure datetime objects can be stored

quantities = np.zeros(data_points)
prices = np.zeros(data_points)
# SKUs = [SKU]*data_points
# names = [name]*data_points

# Generate evenly spaced time instances

for i in range(0,len(dates)):
    int_days = timeintarray[i]
    dates[i] = start_date + datetime.timedelta(days = int_days)

############################
#  STEP 4: RUN SIMULATION  #
############################

#Case 1: Uniform Sales

if case == 1:
    discount_freq = case1_discount_propensity
    for i in range(0,len(dates)):
        qty = np.random.rand()*case1_scale_factor
        discount = round(discount_freq*np.random.rand(),2)
        quantities[i] = round(qty,0)
        prices[i] = MSRP-discount
    
# Case 2: Bimodal Sales  

if case == 2:
    discount_freq = case2_discount_propensity
    for i in range(0,len(dates)):
        discount = round(discount_freq*np.random.rand(),2)
        mon = dates[i].month
        day = dates[i].day
        if (mon == 9 or mon == 12) and day % 2 == 0:
            quantities[i] = np.random.lognormal(0.4,0.25)*case2_scale_factor    
        elif (mon == 9 or mon == 12) and day % 2 != 0:
            quantities[i] = np.random.rand()*(case2_scale_factor-1)
        elif mon == 8 or mon == 11:
            quantities[i] = np.random.lognormal(0.3,0.25)*case2_scale_factor
        elif mon == 3 and day % 2 == 0:
            quantities[i] = np.random.lognormal(0,0.15)*case2_scale_factor
        elif mon == 3 and day % 2 != 0:
            quantities[i] = np.random.rand()*case2_scale_factor*0.6
        else:
            quantities[i] = np.random.rand()*case2_scale_factor
        prices[i] = MSRP-discount  
        
# Case 3: Pronounced Seasonality
# Product experiences low sales for much of the year but exhibits a profound spike in a given season

if case == 3:
    discount_freq = case3_discount_propensity
    for i in range(0,len(dates)):
        discount = round(discount_freq*np.random.rand(),2)
        mon = dates[i].month
        day = dates[i].day
        prices[i] = MSRP - discount
        if case3_season == 1:
            if (mon == 3 or mon == 4 or mon == 5):
                if day % 2 == 0:
                    quantities[i] = abs(math.sin(mon*53)*math.sin(2*day+20))*case3_scale_factor*3*np.random.rand()  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*2
            elif (mon == 2 or mon == 6):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*1.5
            else:
                quantities[i] = np.random.rand()*case3_scale_factor
        elif case3_season == 2:
            if (mon == 6 or mon == 7 or mon == 8):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0.4,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*2
            elif (mon == 5 or mon == 9):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*1.5
            else:
                quantities[i] = np.random.rand()*case3_scale_factor
        elif case3_season == 3:
            if (mon == 9 or mon == 10 or mon == 11):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0.4,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*2
            elif (mon == 8 or mon == 12):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*1.5
            else:
                quantities[i] = np.random.rand()*case3_scale_factor
        elif case3_season == 4:
            if (mon == 12 or mon == 1 or mon == 2):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0.4,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*2
            elif (mon == 11 or mon == 3):
                if day % 2 == 0:
                    quantities[i] = np.random.lognormal(0,0.25)*case3_scale_factor*2  
                else:
                    quantities[i] = np.random.rand()*(case3_scale_factor-1)*1.5
            else:
                    quantities[i] = np.random.rand()*case3_scale_factor
        else:
            raise ValueError("Invalid season entered")
            
# Case 4: Trigonometric Demand Variation

if case == 4:
    discount_freq = case4_discount_propensity
    for i in range(0,len(dates)):
        discount = round(discount_freq*np.random.rand(),2)
        mon = dates[i].month
        day = dates[i].day
        prices[i] = MSRP-discount
        trig_add = 0.6*case4_scale_factor*abs(math.sin(mon*10)*math.sin(day*4))
        quantities[i] = np.random.rand()*case4_scale_factor+trig_add
               
#Case 5: Rising star

if case == 5:
    discount_freq = case4_discount_propensity
    for i in range(0,len(dates)):
        discount = round(discount_freq*np.random.rand(),2)
        days_passed = (dates[i]-start_date).days
        mon = dates[i].month
        day = dates[i].day
        prices[i] = MSRP-discount
        growth = case4_scale_factor*math.exp(days_passed*case5_growth_rate)
        quantities[i] = np.random.rand()*case4_scale_factor+growth

##################################################
#  STEP 5: WRITE RESULTS TO CSV FILE FOR EXPORT  #
##################################################

fwrite = open(outfilename,'w')
for i in range(0,len(dates)):
    line = str(dates[i])+","+SKU+","+name+","+"Sale"+","+str(round(quantities[i],0))+","+str(round(prices[i],2))
    fwrite.write(line)
    fwrite.write('\n')
fwrite.close()

#####################################
#  STEP 6 (OPTIONAL): PLOT RESULTS  #
#####################################

# Uncomment the code below if you wish to see a plot of quantity sold versus time

# plt.scatter(dates,quantities)
# plt.title(f"Plot of quantities sold over time for case {case}")
# plt.xlabel("Date")
# plt.ylabel("Quantity Sold")
  

Enter the SKU to be simulated.  FRG-457-LG-3
Enter the name of the product to be simulated.  LG Smart Fridge
CHOOSE CASE 3 SEASON: Enter 1 for spring, 2 for summer, 3 for fall, or 4 for winter:  4
Enter the output file name.  fridge.csv


In [ ]:
""" Garbage code
    # rand_days = round(np.random.rand()*num_days,0)

    # print(f"The product name is {name}. It corresponds to SKU {SKU}.")

    # np.sort(times)
    # print(times)

    # yvals = np.linspace(0,5,data_points)

    # plt.scatter(times, yvals)
    
            # if dates[i].month() == "November":
        #     quantities[i] = 400
        
        # if dates[i] > start_date + datetime.timedelta(days = 55):
            # quantities[i] = 400
        # else:
            # quantities[i] = 50
    
    # print(SKUs)
    
    day = dates[52]
print(day.month)
print(type(dates[1]))
# # plt.scatter(dates,prices,color = 'r')
# plt.legend(["Quantities Sold", "Sale Prices ($)"])

# date = datetime.date(2022,3,1)
# date += datetime.timedelta(days = 50)
# print(date)

# you can use datetime's timedelta method to add time to a date
# this means you can simulate a random number of days between two start and end date!

# help(np.sort)

"""